In [24]:
import pandas as pd
import numpy as np
import os

### Query Data

In [18]:
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

query = """
SELECT 
  yeti_customer_uuid,
  COUNT( CASE WHEN web_action = 'product_added' THEN web_action ELSE NULL END) AS product_added_count,
  COUNT( CASE WHEN web_action = 'product_added_to_wishlist' THEN web_action ELSE NULL END) AS product_added_to_wishlist_count,
  COUNT( CASE WHEN web_action = 'product_removed' THEN web_action ELSE NULL END) AS product_removed_count,
  COUNT( CASE WHEN web_action = 'customizer_started' THEN web_action ELSE NULL END) AS customizer_started_count,
  COUNT( CASE WHEN web_action = 'customizer_completed' THEN web_action ELSE NULL END) AS customizer_completed_count,
  COUNT (CASE WHEN web_action = 'cart_viewed' THEN web_action ELSE NULL END) AS cart_viewed_count,
  COUNT( CASE WHEN web_action = 'checkout_started' THEN web_action ELSE NULL END) AS checkout_started_count,
  COUNT( CASE WHEN web_action = 'order_cancelled' THEN web_action ELSE NULL END) AS order_cancelled_count

FROM  `yeti-dev-edw.aa_anon.aa_web_action` web 
WHERE source_country = 'US' 
GROUP BY yeti_customer_uuid

"""
job = client.query(query)
df = job.to_dataframe()


In [25]:
os.getcwd()

'/home/jupyter/Data Engineering'

#### Import data_with_entry_prod_cat.csv

In [33]:
data = pd.read_csv('/home/jupyter/data_with_entry_prod_cat.csv')

In [28]:
data = data.iloc[:,1:]

#### Change name of queried data

In [34]:
web_action_df = df

In [35]:
web_action_df

,yeti_customer_uuid,product_added_count,product_added_to_wishlist_count,product_removed_count,customizer_started_count,customizer_completed_count,cart_viewed_count,checkout_started_count,order_cancelled_count
0,c8bd7c44-523a-41b4-9163-e2e4e97ae568,61,0,25,45,36,72,20,0
1,13aba6c8-c83d-46b6-8838-4b56043f6bc0,37,0,3,3,0,26,24,0
2,a3326e55-c13d-4af6-9154-033d6d2aaee0,58,3,27,48,17,94,31,0
3,d331747f-92a8-44e5-80b9-16acdd5012ee,34,0,3,32,33,11,5,0
4,fa75003d-0cf9-4150-b1f7-e52bea3c8a2e,273,2,370,9,1,94,67,0
...,...,...,...,...,...,...,...,...,...
5510081,b8533384-b05c-4576-9653-234826e4aa91,15,0,5,1,0,53,1,0
5510082,0f755a22-debb-42c2-9d77-4f7d7e744260,15,0,14,20,15,13,7,0
5510083,d9367fc6-43e3-4565-92f3-6915b052e7d5,15,0,1,17,11,14,10,0
5510084,167c0d03-3c0e-40f2-a1ad-e62ec20cc1da,15,0,3,18,13,11,3,0


#### Merge the two data frames (left join on customer id)

In [45]:
data_merged = pd.merge(data, web_action_df, how = 'left', left_on = 'customer_id', right_on = 'yeti_customer_uuid')

In [48]:
data_merged = data_merged.iloc[:,1:]

In [50]:
data_merged.isna().sum()

customer_id                             0
months_elapsed                          0
lifetime_spent                          0
per_order_spent                         0
outdoor_equipment_pct                2594
Cargo_pct                            2594
Soft_Cooler_pct                      2594
Bags_pct                             2594
drinkware_pct                        2594
Hard_Cooler_pct                      2594
other_pct                            2594
purchase_times                          0
recency                                 0
avg_time_btw_orders                     0
email_received                          0
email_open_rate                    383304
unsubscribed                       383304
months_since_last_email_open       545492
num_distinct_prod_cats                  0
lifetime_quantity                       5
per_order_quantity                      5
has_discount                            5
black                               11837
navy                              

#### Exporting this to CSV

In [51]:
data_merged.to_csv('merged_data_03_15_23.csv')